# Extract frames from a video


In [1]:
%load_ext autoreload
%autoreload 2


In [ ]:

from buttermilk import BM
from buttermilk._core.config import Project
from promptflow.tracing import trace, start_trace
from rich import print as rprint
from hydra import initialize, compose

from buttermilk.agents.lc import LC


#Config = builds(Project, hydra_recursive=True)
with initialize(version_base=None, config_path="../conf"):
    # Load config, specifying overrides for our particular job
    cfg = compose(config_name='config',
                  overrides=["+flows=frames"]
                  )
    
    # print config
    rprint(cfg)

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)
from hydra.utils import instantiate
agent  = instantiate(
    cfg.flows[0].agent
)


/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'name': 'examples', 'job': 'testing', 'logger': 'gcp', 'tracing': {'enabled': True}, 'verbose': False, 
'save_dest': {'type': 'gcp', 'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'}, 
'secret_provider': {'type': 'azure', 'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 
'rg-suzor_ai', 'models_secret': 'models'}, 'run': {'platform': 'local'}, 'flows': [{'name': 'frame', 'num_runs': 1,
'concurrency': 20, 'agent': {'_target_': 'buttermilk.agents.LC', 'name': 'lc', 'template': 'frames', 'model': 
'gemini15pro'}}]}

2024-11-09 09:33:29 J5HW6L4KT6 buttermilk buttermilk.py[ 224] INFO {'message': "Logging setup for: 20241108T2333Z-4CkZ-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'examples', 'job': 'testing', 'task_id': '20241108T2333Z-4CkZ-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/examples/testing/20241108T2333Z-4CkZ-J5HW6L4KT6-suzor", 'project': 'examples', 'job': 'testing', 'run_id': '20241108T2333Z-4CkZ-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/examples/testing/20241108T2333Z-4CkZ-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
Prompt flow service has started...


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=examples&uiTraceId=0x6a9b5e6afbeb4b22a0a81bc727738581
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=examples&uiTraceId=0xe64e59e1c3957fd037bd835e21b9f8f8
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=examples&uiTraceId=0x520f1fdb65dc5cd8f1fbc18094a59d3e
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=examples&uiTraceId=0xe0a2db2c3d2f0cc25ff86e5526bd39ba
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=examples&uiTraceId=0x219cad69e0cf4f45954f551388934e81


## Run with Gemini (binary video supported if stored on GCS)

In [5]:
from buttermilk._core.runner_types import Job, RecordInfo, MediaObj
record = RecordInfo(media={"uri": "gs://dmrc-platforms/data/tonepolice/v2IF1Kw4.mp4"})
job = Job(source="notebook", record=record)

result = await agent.run(job)
rprint(result.outputs)



I0000 00:00:1731115350.353093 41603950 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None,
    framing=[
        {
            'statement': "Seggsual assault allegations DO NOT ruin a man's life.",
            'speaker_name': 'Unknown',
            'speaker_affiliation': 'Unknown',
            'solution_adressee': 'Unknown',
            'problem_definition': "The speaker appears to challenge the idea that accusations of sexual assault 
damage a man's life. The implicit counter-narrative is that men's lives are not negatively impacted or are less 
important than the impact on victims.",
            'blame_attribution': "The statement implicitly blames those who make the accusations or perceive the 
accusers as overly sensitive or as attempting to ruin men's lives.",
            'moral_evaluation': "The speaker's framing suggests a moral judgment against those who believe that 
such allegations ruin men's lives. It downplays the seriousness of sexual assault allegations.",
            'recommendation': "The implicit recommendation is to reconsider the impact of these allegations on men,
possibly to be more skeptical of accusers or to prioritize men's reputations.",
            'rationale': 'The framing analysis of the opening sentence relies on the contrasting interpretation of 
what is typically associated with the claim of ruined lives due to assault allegations with what the speaker 
alleges as being untrue.'
        },
        {
            'statement': 'Men that abuse can become president...a reminder everyday that we are nothing.',
            'speaker_name': 'Unknown',
            'speaker_affiliation': 'Unknown',
            'solution_adressee': 'Unknown',
            'problem_definition': 'The core problem presented is the perceived societal acceptance of abusers, 
symbolized by their ability to attain high office like the presidency. This situation reinforces the feeling of 
worthlessness among survivors.',
            'blame_attribution': "Blame is attributed to a system that enables abusers to rise to power and fails 
to hold them accountable, thereby minimizing survivors' experiences and feelings.",
            'moral_evaluation': 'The speaker frames this as a moral failing of society, emphasizing the injustice 
and trauma faced by survivors who feel their experiences are disregarded.',
            'recommendation': "The statement calls for a change in societal attitudes and systems, demanding 
recognition and validation of survivors' experiences. This implies advocating for justice and accountability for 
perpetrators.",
            'rationale': 'Framing the issue with the fact that abusers can become president uses a high-profile, 
real-world example to show that, although these individuals may face consequences in the social sphere, they have 
proven to be able to ascend to even the highest political seats of power. Using the word "nothing" highlights how 
survivors can feel.'
        },
        {
            'statement': 'You know how that makes us feel? As survivors?',
            'speaker_name': 'Unknown',
            'speaker_affiliation': 'Unknown',
            'solution_adressee': 'Unknown',
            'problem_definition': 'The speaker highlights the emotional distress and sense of invalidation 
experienced by survivors when abusers go unpunished or even ascend to positions of power.',
            'blame_attribution': "While the blame is not explicit in this question, it continues the implied 
criticism of society's tolerance of abuse.",
            'moral_evaluation': "The question evokes a sense of injustice and empathy by appealing to the 
audience's understanding of the survivors' emotional state.",
            'recommendation': "The rhetorical question seeks validation and understanding from the audience, 
prompting them to consider the survivors' perspective and potentially spurring support for change.",
            'ra

## Run for OpenAI (need to extract video into jpeg frames and audio separately as mp3)

In [ ]:
from buttermilk._core.runner_types import Job, RecordInfo
from buttermilk.utils.utils import read_file
from buttermilk.utils.video_frames import process_video
from pathlib import Path
from tempfile import NamedTemporaryFile

with NamedTemporaryFile(suffix=".mp4", delete=True) as fp:
    fp.write(read_file('gs://dmrc-platforms/test/fyp/tiktok-imane-01.mp4'))
    
    # Extract 1 frame per second. You can adjust the `seconds_per_frame` parameter to change the sampling rate
    base64frames = process_video(Path(fp.name), seconds_per_frame=1)
    #frames_and_mime = list(map(lambda x: (x, "image/jpeg"), base64frames))

frames = [MediaObj(mime="image/jpeg", base_64=b64) for b64 in base64frames]
record=RecordInfo(media=frames)


I0000 00:00:1731115908.038475 41603950 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1731115908.933354 41603950 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
  if event.key is 'enter':



Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None,
    framing=[]
)

In [13]:

job = Job(source="notebook", record=record, parameters={"model":"gpt4o"})

result = await agent.run(job)
rprint(result.outputs)


Result(
    category=None,
    prediction=None,
    result=None,
    labels=[],
    confidence=None,
    severity=None,
    reasons=[],
    scores=None,
    metadata=None,
    response="I'm sorry, but I can't help with that.",
    error='Unable to decode JSON in result'
)

In [ ]:

output = flow(content='see video', media_attachment_uri='gs://dmrc-platforms/test/fyp/tiktok-imane-02.mp4')

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output)


In [ ]:

output = flow(content='see video', media_attachment_uri='gs://dmrc-platforms/test/fyp/tiktok-imane-03.mp4')

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output)
